In [1]:
import sys
sys.path.append('../')
sys.path.append('../utils/')
import schema
import datajoint_utils as dju
import database_pop as pop
import numpy as np

import importlib

[2025-06-24 19:15:17,414][INFO]: Connecting root@127.0.0.1:3306
[2025-06-24 19:15:17,515][INFO]: Connected root@127.0.0.1:3306


/Users/riekelabbackup/Desktop/Vyom/RetinAnalysis/utils/config.ini


In [2]:
importlib.reload(pop)
# pop.reload_celltypefiles()

<module 'database_pop' from '/Users/riekelabbackup/Desktop/Vyom/RetinAnalysis/nbs/../utils/database_pop.py'>

In [3]:
importlib.reload(dju)
protocol_names = dju.search_protocol('dots')
df = dju.get_datasets_from_protocol_names(protocol_names)
df.head()

Found 1 protocols matching "dots":
['manookinlab.protocols.ObjectMotionDots']
Found 60 experiments, 107 epoch blocks.


,exp_name,datafile_name,NDF,chunk_name,protocol_name,is_mea,data_dir,group_label,experiment_id,protocol_id,group_id,block_id,chunk_id
0,20230614C,data010,0.0,chunk2,manookinlab.protocols.ObjectMotionDots,1,20230614C/data010,OMS dots,2,13,19,19,2
1,20230713C,data030,2.0,chunk2,manookinlab.protocols.ObjectMotionDots,1,20230713C/data030,Mesopic OMS dots,4,13,62,67,6
2,20230713C,data031,0.0,chunk2,manookinlab.protocols.ObjectMotionDots,1,20230713C/data031,Photopic OMS dots,4,13,63,68,6
3,20230719C,data026,0.0,chunk3,manookinlab.protocols.ObjectMotionDots,1,20230719C/data026,oms dots,5,13,85,96,9
4,20230719C,data027,0.0,chunk3,manookinlab.protocols.ObjectMotionDots,1,20230719C/data027,oms dots,5,13,85,97,9


In [4]:
import stim
importlib.reload(stim)
importlib.reload(dju)
idx = 0
ls_const = ['exp_name', 'datafile_name']
s1 = stim.Stim(**df.loc[idx,ls_const].to_dict())
s1

Stim with properties:
  exp_name: 20230614C
  datafile_name: data010
  chunk_name: chunk2
  protocol_name: manookinlab.protocols.ObjectMotionDots
  noise_protocol_name: manookinlab.protocols.FastNoise
  ss_version: kilosort2.5

In [5]:
# Need to find list of noise chunks sorted by time distance to protocol
importlib.reload(dju)
df_exp = dju.get_mea_exp_summary(s1.exp_name)
df_exp.head()

,exp_name,datafile_name,group_label,NDF,chunk_name,protocol_name,duration_minutes,minutes_since_start,start_time,end_time,data_dir,experiment_id,group_id,block_id,chunk_id,protocol_id
0,20230614C,data000,noise,0.0,chunk1,manookinlab.protocols.FastNoise,61.38,61.38,2023-06-14 11:16:26,2023-06-14 12:17:49,20230614C/data000,2,9,9,1,17
1,20230614C,data001,delta size noise,0.0,chunk1,manookinlab.protocols.NoiseSizeModulation,5.62,69.28,2023-06-14 12:20:06,2023-06-14 12:25:43,20230614C/data001,2,10,10,1,18
2,20230614C,data002,delta time noise,0.0,chunk1,manookinlab.protocols.NoiseSizeModulation,5.03,76.38,2023-06-14 12:27:47,2023-06-14 12:32:49,20230614C/data002,2,11,11,1,18
3,20230614C,data003,crf,0.0,chunk1,manookinlab.protocols.ContrastResponseGrating,1.22,78.58,2023-06-14 12:33:48,2023-06-14 12:35:01,20230614C/data003,2,12,12,1,5
4,20230614C,data004,adapt noise spatial,0.0,chunk1,manookinlab.protocols.AdaptNoiseColorSteps,15.45,101.87,2023-06-14 12:42:51,2023-06-14 12:58:18,20230614C/data004,2,13,13,1,19


In [6]:
importlib.reload(dju)
df_typing = dju.get_typing_files_for_protocol(s1.exp_name, s1.datafile_name, s1.noise_protocol_name, True)
df_typing

Found 4 noise chunks for protocol "manookinlab.protocols.FastNoise"
Noise chunks sorted by distance:
  chunk2: 130.82 minutes
  chunk1: 34.27 minutes
  chunk3: 193.83 minutes
  chunk4: 430.78 minutes
Found 1 cell typing file(s) for chunk2
Found 1 cell typing file(s) for chunk2
Found 1 cell typing file(s) for chunk2
Found 1 cell typing file(s) for chunk2


,exp_name,datafile_name,datafile_names,chunk_name,protocol_name,ss_version,typing_file_name,typing_file_path
0,20230614C,data012,[data012],chunk2,manookinlab.protocols.FastNoise,kilosort2,kilosort2.classification-FMR.txt,/Volumes/data-1/analysis/20230614C/chunk2/kilo...


In [8]:
importlib.reload(stim)
ls_const = ['exp_name', 'datafile_name', 'ss_version']
idx = 0
n1 = stim.NoiseStim(**df_typing.loc[idx,ls_const].to_dict())
n1

NoiseStim with properties:
  exp_name: 20230614C
  datafile_name: data012
  chunk_name: chunk2
  protocol_name: manookinlab.protocols.FastNoise
  noise_protocol_name: manookinlab.protocols.FastNoise
  ss_version: kilosort2
  staXChecks: 100
  staYChecks: 75
  numXChecks: 100
  numYChecks: 75
  microns_per_pixel: 3.8
  canvas_size: [800, 600]
  data_files: ['data012']

In [ ]:
importlib.reload(dju)
protocol_names = dju.search_protocol('spatialnoise')
df = dju.get_datasets_from_protocol_names(protocol_names)
df.head()

Found 4 protocols matching "spatialnoise":
['manookinlab.protocols.DisplacedSpatialNoise'
 'manookinlab.protocols.FlashedSpatialNoise'
 'manookinlab.protocols.SpatialNoise'
 'manookinlab.protocols.SpatialNoiseAndMotion']
No epochs found for block 2372
Found 48 experiments, 192 epoch blocks.


,exp_name,datafile_name,NDF,chunk_name,protocol_name,is_mea,data_dir,group_label,experiment_id,protocol_id,group_id,block_id,chunk_id
0,20231003C,data003,0.5,chunk1,manookinlab.protocols.SpatialNoise,1,20231003C/data003,noise,68,3,1499,2285,208
1,20231003C,data004,0.5,chunk1,manookinlab.protocols.SpatialNoise,1,20231003C/data004,noise,68,3,1499,2286,208
2,20231003C,data005,0.5,chunk1,manookinlab.protocols.SpatialNoise,1,20231003C/data005,noise,68,3,1499,2287,208
3,20231003C,data017,0.5,chunk3,manookinlab.protocols.SpatialNoise,1,20231003C/data017,spatial noise,68,3,1506,2296,206
4,20231003C,data027,0.5,chunk4,manookinlab.protocols.SpatialNoise,1,20231003C/data027,spatial noise,68,3,1516,2306,207


In [44]:
df[df['group_label'].str.contains('big')]

,exp_name,datafile_name,NDF,chunk_name,protocol_name,is_mea,data_dir,group_label,experiment_id,protocol_id,group_id,block_id,chunk_id
20,20231220C,data007,0.5,big_noise,manookinlab.protocols.SpatialNoise,1,20231220C/data007,big noise,73,3,1580,2380,228
25,20240117C,data009,0.5,big_noise,manookinlab.protocols.SpatialNoise,1,20240117C/data009,big noise,74,3,1607,2410,234
32,20240130C,data025,0.5,big_noise,manookinlab.protocols.SpatialNoise,1,20240130C/data025,big noise,75,3,1651,2456,240
33,20240130C,data026,0.5,big_noise,manookinlab.protocols.SpatialNoise,1,20240130C/data026,big noise,75,3,1651,2457,240
146,20250219H,data018,0.0,chunk3,manookinlab.protocols.SpatialNoise,1,20250219H/data018,big noise,120,3,2187,3301,426
147,20250219H,data019,0.0,chunk3,manookinlab.protocols.SpatialNoise,1,20250219H/data019,big noise,120,3,2187,3302,426
164,20250306C,data005,0.0,big_noise,manookinlab.protocols.SpatialNoise,1,20250306C/data005,big noise,127,3,2296,3487,477
